#### 1. Install Dependencies

In [ ]:
pip install -r ../requirements.txt -q

In [ ]:
pip show langchain openai

#### 2. Verify python-dotenv

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

os.environ.get("OPENAI_API_KEY")

#### 3. LLM Text Models

In [ ]:
# init text based llm
from langchain import OpenAI
llm = OpenAI(model_name="text-davinci-003", temperature=0.7, max_tokens=512)
print(llm)

# get no. of tokens
print('\n\nnum tokens: ' + str(llm.get_num_tokens('tell me a joke')))  
  
# get output for one question
output1 = llm('explain java in one sentence')
print('\n\noutput1: ' + output1)

# get output for multiple questions
output2 = llm.generate(['what is the capital of India', 'explain c# in one sentence'])
print('no. of answers: ' + str(len(output2.generations)))
print(output2)
print('\n1st answer: ' + output2.generations[0][0].text)
print('\n2nd answer: ' + output2.generations[1][0].text)

# get multiple answers for a single question
output3 = llm.generate(['Tell me a one liner joke'] * 3)
for o in output3.generations:
  print('\n\njoke: ' + o[0].text)

#### 4. LLM Chat Models

In [ ]:
# init chat based llm
from langchain.schema import (AIMessage, SystemMessage, HumanMessage)
from langchain.chat_models import  ChatOpenAI
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.5, max_tokens=1024)
print(llm)

# start chatting
messages = [
  # set context for our assistant
  SystemMessage(content='You are doctor in India and respond only in Hindi'),
  # ask a question as a user
  HumanMessage(content='I have fever. What off the shelf medicine should I take? Answer iun one line please'),
]
output = llm(messages)
print(output.content)

#### 5. Prompt Templates

In [ ]:
from langchain import PromptTemplate
from langchain import OpenAI

template = '''
You are an experienced doctor. Write a one line answer for {disease} in {language}
'''

prompt = PromptTemplate(
  input_variables=['disease', 'language'], 
  template=template
)
print(prompt)

llm = OpenAI(model_name="text-davinci-003", temperature=0.7, max_tokens=512)
print(llm)

output = llm(prompt.format(disease='diabetes', language='english'))
print(output)
output = llm(prompt.format(disease='jaundice', language='hindi'))
print(output)

#### 6. Simple Chains

In [ ]:
from langchain import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain

# create prompt
template = '''
You are an experienced doctor. Write a one line answer for {disease} in {language}
'''
prompt = PromptTemplate(
  input_variables=['disease', 'language'], 
  template=template
)

# create llm
llm = OpenAI(model_name="text-davinci-003", temperature=0.7, max_tokens=512)

#  create chain
chain = LLMChain(llm=llm, prompt=prompt)

#  run chain
output = chain.run({'disease': 'diabetes', 'language': 'english'})
print(output)
output = chain.run({'disease': 'jaundice', 'language': 'hindi'})
print(output)
output = chain.run({'disease': 'fever', 'language': 'german'})
print(output)

# if the prompt had just one variable, we could simply do this
# output = chain.run('fever')

#### 7. Sequential Chains

In [ ]:
from langchain import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
from langchain.chains import LLMChain, SimpleSequentialChain

# create llm 1
llm_1 = OpenAI(model_name="text-davinci-003", temperature=0.7, max_tokens=512)
prompt_1 = PromptTemplate(
  input_variables=['concept'], 
  template='''You are an experienced python programmer. Write a simple easy to understand function that implements the concept of {concept}'''
)
chain_1 = LLMChain(llm=llm_1, prompt=prompt_1)

# create llm 2
llm_2 = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.7, max_tokens=512)
prompt_2 = PromptTemplate(
  input_variables=['function'], 
  template='''Given a python function {function}, describe it in detail'''
)
chain_2 = LLMChain(llm=llm_2, prompt=prompt_2)

# create sequential chain
chain = SimpleSequentialChain(chains=[chain_1, chain_2], verbose=True)

# run chain
output = chain.run('recursion')

#### 8. LangChain Agents

In [ ]:
from langchain_experimental.agents.agent_toolkits import create_python_agent
from langchain_experimental.tools.python.tool import PythonREPLTool
from langchain.llms import OpenAI

llm = OpenAI(temperature=0)
agent_executor = create_python_agent(
  llm=llm, 
  tool=PythonREPLTool(description='put the final command inside print()'),
  verbose=True,
)
agent_executor.run('calculate the square root of factorial of 20 and display it in 3 decimal points')
agent_executor.run('what is the answer to 5.1 ** 7.3?')